In [1]:
from transformers import (
    BertModel, 
    AutoConfig, 
    AutoTokenizer, 
    Trainer,
    TrainingArguments
    )
import torch.nn as nn
import datasets
import csv


In [2]:
import torch
import numpy as np

torch.cuda.is_available()

True

In [3]:
import csv

In [4]:
## Load dataset
feats_fp = open("BERT_X.csv", "r")
labels_fp = open("BERT_y.csv", "r")
feats = csv.reader(feats_fp)
labels = csv.reader(labels_fp)

# skip header
next(feats)
next(labels)

unscaled_data = {'text': [], 'label': []}
nn_data = []
for row in feats:
    unscaled_data['text'].append(row[0].strip().replace("\n", " "))
    nn_feats = [float(col.strip().replace("\n", "")) for col in row[1:]]
    
    
    nn_data.append(nn_feats[-2]) # company employee count only
for i,row in enumerate(labels):
    unscaled_data['label'].append(float(row[0].strip().replace("\n", "")))
nn_data = np.array(nn_data).reshape(-1,1).astype(np.float32)
nn_data[:5]

array([[1171.],
       [  36.],
       [1227.],
       [1260.],
       [  36.]], dtype=float32)

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_labels = scaler.fit_transform(np.array(unscaled_data['label']).reshape(-1,1)).flatten()
scaled_data = unscaled_data
scaled_data['label'] = scaled_labels

employee_count_scaler = StandardScaler()
scaled_employee_count =employee_count_scaler.fit_transform(nn_data[:,0].reshape(-1,1)).flatten()
nn_data[:,0] = scaled_employee_count
scaled_labels, nn_data[:,0]

(array([-0.58786439, -0.94649659,  2.86819603, ..., -1.04441134,
        -0.86218111, -0.69471581]),
 array([-0.30006495, -0.31242564, -0.2994551 , ..., -0.31238207,
        -0.3128286 , -0.31278503], dtype=float32))

In [6]:
assert len(scaled_data['text']) == len(scaled_data['label']) == nn_data.shape[0]
dataset = datasets.Dataset.from_dict(scaled_data)

In [7]:
print(dataset[0]['text'])
scaler.inverse_transform(np.array([dataset[0]['label']]).reshape(-1,1))

Overview  HearingLife is a national hearing care company and part of the Demant Group, a global leader in hearing healthcare built on a heritage of care, health, and innovation since 1904. HearingLife operates more than 600 hearing care centers across 42 states. We follow a scientific, results-oriented approach to hearing healthcare that is provided by highly skilled and caring professionals. Our vision is to help more people hear better through life-changing hearing health delivered by the best personalized care. This Team Member must uphold the HearingLife Core Values:   We create trust  We are team players  We apply a can-do attitude  We create innovative solutions   Responsibilities  You will help more people hear better by providing clinical expertise to diagnose and treat hearing loss while ensuring a positive patient experience. The Hearing Care Provider acts in accordance with required industry and state professional licensing standards and local practice scope and is responsib

array([[63000.]])

In [8]:
# Tokenize the dataset
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
def preprocess(examples):
    return tokenizer(examples['text'], truncation=True, padding=True)

tokenized_dataset = dataset.map(preprocess, batched=True)

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

  0%|          | 0/27 [00:00<?, ?ba/s]

In [9]:
# training_args = TrainingArguments(
#     output_dir='./results',          
#     num_train_epochs=3,              
#     per_device_train_batch_size=16,  
#     learning_rate=5e-5,               
#     warmup_steps=500,                
#     weight_decay=0.01,              
#     logging_dir='./logs',
# )


In [10]:
tokenized_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 26990
})

In [11]:
from torch.utils.data import Dataset

class RegressionDataset(Dataset):
    def __init__(self, input_ids, attention_mask, labels, nn_data):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels
        self.nn_data = nn_data

    def __getitem__(self, idx):
        item = {
            'input_ids': torch.tensor(self.input_ids[idx]),
            'attention_mask': torch.tensor(self.attention_mask[idx]),
            'labels': torch.tensor(self.labels[idx]),
            'nn_data': torch.tensor(self.nn_data[idx])
        }
        return item

    def __len__(self):
        return len(self.labels)  # Assuming all data entries have labels

    
    

In [12]:
input_ids = tokenized_dataset['input_ids']
attention_mask = tokenized_dataset['attention_mask']
labels = dataset['label']  # Assuming your labels are in the original dataset
nn_data = nn_data

reg_dataset = RegressionDataset(input_ids, attention_mask, labels, nn_data)
dataloader = torch.utils.data.DataLoader(reg_dataset, batch_size=64, shuffle=True)



In [13]:
reg_dataset[0]['nn_data']

tensor([-0.3001])

In [14]:

model_name = "bert-base-uncased"
config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
bert_model = BertModel.from_pretrained(model_name)

config.problem_type = 'regression'

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:

# Add linear layer
output_size = 1  


# Combine BERT and the linear layer
class BertWithLinear(nn.Module):
    def __init__(self):
        super(BertWithLinear, self).__init__()
        self.bert = bert_model.to('cuda')
    
        self.nn = nn.Sequential(
            nn.Linear(config.hidden_size + len(nn_data[0]) , 128),
            nn.ReLU(),
            nn.Linear(128, 16),
            nn.ReLU(),
            nn.Linear(16, output_size)
        ).to('cuda')
        

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None, inputs_embeds=None, labels=None, nn_data = None):
        output = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, position_ids=position_ids, head_mask=head_mask, inputs_embeds=inputs_embeds)
        # Use pooled output for classification/regression
        pooled_output = output.pooler_output
        nn_data = torch.cat((pooled_output, nn_data), 1)
        nn_output = self.nn(nn_data)
        return nn_output

    

model = BertWithLinear().to('cuda')

In [18]:
# freeze/unfreeze BERT pretrained weights
for param in model.bert.embeddings.parameters():
    param.requires_grad = True
model = nn.DataParallel(model)

In [19]:
model.parameters()

<generator object Module.parameters at 0x2baa3d731308>

In [20]:
loss_fn = nn.MSELoss()  # Mean Squared Error is common for regression
optimizer = torch.optim.Adam([
    {'params': model.module.bert.parameters(), 'lr': 1e-5}, 
    {'params': model.module.nn.parameters(), 'lr': 2e-5} # our neural net
])
num_epochs = 10
    
    
from torch.utils.data import random_split 

train_size = int(0.8 * len(reg_dataset))  # 80% of the dataset for training
val_size = len(reg_dataset) - train_size 

# Create the train and validation datasets
train_dataset, val_dataset = random_split(reg_dataset, [train_size, val_size])

# Create DataLoaders
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)  # No need to shuffle validation
train_losses = []
val_losses = []
# Modify your training loop (add validation)
for epoch in range(num_epochs):
    print(epoch,"/",num_epochs)
    b=0
    ## Training Phase
    model.train()  # Set model to training mode
    current_epoch_train_losses = []
    for batch in train_dataloader:
        input_ids = batch['input_ids'].to('cuda') 
        attention_mask = batch['attention_mask'].to('cuda')
        labels = batch['labels'].to('cuda')
        data = batch['nn_data'].to('cuda')

        optimizer.zero_grad()
        
        outputs = model(input_ids, attention_mask=attention_mask, nn_data=data)
        
        loss = loss_fn(outputs.squeeze(1), labels) # Ensure outputs are single-dimensional
        current_epoch_train_losses.append(loss.item())
        loss.backward()
        optimizer.step() 
        print(f"batch {b} complete. Loss: ",loss.item())
        b+=1
    train_losses.append(sum(current_epoch_train_losses)/len(current_epoch_train_losses))
    ## Validation Phase
    model.eval()   # Set model to evaluation mode
    val_loss = 0
    with torch.no_grad():  # Disable gradient calculation for validation
        for batch in val_dataloader:
            input_ids = batch['input_ids'].to('cuda') 
            attention_mask = batch['attention_mask'].to('cuda')
            labels = batch['labels'].to('cuda')
            data = batch['nn_data'].to('cuda')
            outputs = model(input_ids, attention_mask=attention_mask,nn_data=data)
            val_loss += loss_fn(outputs.squeeze(1), labels).item()

    val_loss /= len(val_dataloader)
    val_losses.append(val_loss)
    print(f"Epoch {epoch} Validation Loss: {val_loss}")

    # checkpoint model
    if epoch % 5 == 0:
        torch.save(model.state_dict(), f"model_checkpoint_{epoch}.pth")


0 / 10
batch 0 complete. Loss:  0.9602559208869934
batch 1 complete. Loss:  1.0854591131210327
batch 2 complete. Loss:  0.9973961114883423
batch 3 complete. Loss:  1.4335261583328247
batch 4 complete. Loss:  1.3671677112579346
batch 5 complete. Loss:  1.295344352722168
batch 6 complete. Loss:  0.7374680638313293
batch 7 complete. Loss:  1.401750087738037
batch 8 complete. Loss:  0.7906998991966248
batch 9 complete. Loss:  0.8172276020050049
batch 10 complete. Loss:  1.7089563608169556
batch 11 complete. Loss:  0.8064703941345215
batch 12 complete. Loss:  0.7174537181854248
batch 13 complete. Loss:  0.865260124206543
batch 14 complete. Loss:  0.5848900079727173
batch 15 complete. Loss:  1.352117657661438
batch 16 complete. Loss:  0.954603374004364
batch 17 complete. Loss:  0.8742499351501465
batch 18 complete. Loss:  0.6452634334564209
batch 19 complete. Loss:  0.82177734375
batch 20 complete. Loss:  0.9450459480285645
batch 21 complete. Loss:  0.6680349111557007
batch 22 complete. Loss

batch 181 complete. Loss:  0.39483755826950073
batch 182 complete. Loss:  0.5694147348403931
batch 183 complete. Loss:  0.4627424478530884
batch 184 complete. Loss:  0.2930580973625183
batch 185 complete. Loss:  0.1919148564338684
batch 186 complete. Loss:  0.3649362325668335
batch 187 complete. Loss:  0.3447125256061554
batch 188 complete. Loss:  0.47085410356521606
batch 189 complete. Loss:  0.5182582139968872
batch 190 complete. Loss:  0.43994611501693726
batch 191 complete. Loss:  0.23894815146923065
batch 192 complete. Loss:  0.6318080425262451
batch 193 complete. Loss:  0.39887145161628723
batch 194 complete. Loss:  0.3728852868080139
batch 195 complete. Loss:  0.25238123536109924
batch 196 complete. Loss:  0.2812676727771759
batch 197 complete. Loss:  0.34678804874420166
batch 198 complete. Loss:  0.5716134309768677
batch 199 complete. Loss:  0.23236222565174103
batch 200 complete. Loss:  0.6836685538291931
batch 201 complete. Loss:  0.2768794298171997
batch 202 complete. Loss: 

batch 358 complete. Loss:  0.35365960001945496
batch 359 complete. Loss:  0.46692293882369995
batch 360 complete. Loss:  0.2560327649116516
batch 361 complete. Loss:  0.24280674755573273
batch 362 complete. Loss:  0.24585723876953125
batch 363 complete. Loss:  0.2919696867465973
batch 364 complete. Loss:  0.19065041840076447
batch 365 complete. Loss:  0.5062116384506226
batch 366 complete. Loss:  0.20257794857025146
batch 367 complete. Loss:  0.2734726369380951
batch 368 complete. Loss:  0.16914090514183044
batch 369 complete. Loss:  0.25578540563583374
batch 370 complete. Loss:  0.18478341400623322
batch 371 complete. Loss:  0.2783803343772888
batch 372 complete. Loss:  0.18059737980365753
batch 373 complete. Loss:  0.20651701092720032
batch 374 complete. Loss:  0.2137545496225357
batch 375 complete. Loss:  0.5165082812309265
batch 376 complete. Loss:  0.3242429494857788
batch 377 complete. Loss:  0.24461817741394043
batch 378 complete. Loss:  0.19612160325050354
batch 379 complete. L

batch 535 complete. Loss:  0.18408045172691345
batch 536 complete. Loss:  0.2844407558441162
batch 537 complete. Loss:  0.35551685094833374
batch 538 complete. Loss:  0.18453240394592285
batch 539 complete. Loss:  0.39302054047584534
batch 540 complete. Loss:  0.5946670174598694
batch 541 complete. Loss:  0.256773442029953
batch 542 complete. Loss:  0.17932510375976562
batch 543 complete. Loss:  0.2490367293357849
batch 544 complete. Loss:  0.6271642446517944
batch 545 complete. Loss:  0.3792073726654053
batch 546 complete. Loss:  0.3869614601135254
batch 547 complete. Loss:  0.3545866012573242
batch 548 complete. Loss:  0.3681737184524536
batch 549 complete. Loss:  0.7042070627212524
batch 550 complete. Loss:  0.23133337497711182
batch 551 complete. Loss:  0.29018229246139526
batch 552 complete. Loss:  0.363040566444397
batch 553 complete. Loss:  0.2969742715358734
batch 554 complete. Loss:  0.3672666847705841
batch 555 complete. Loss:  0.16351954638957977
batch 556 complete. Loss:  0

batch 36 complete. Loss:  0.2930046319961548
batch 37 complete. Loss:  0.23612724244594574
batch 38 complete. Loss:  0.1264781504869461
batch 39 complete. Loss:  0.09751620888710022
batch 40 complete. Loss:  0.2730185389518738
batch 41 complete. Loss:  0.17502833902835846
batch 42 complete. Loss:  0.32565608620643616
batch 43 complete. Loss:  0.22074949741363525
batch 44 complete. Loss:  0.2422146499156952
batch 45 complete. Loss:  0.14602452516555786
batch 46 complete. Loss:  0.17501334846019745
batch 47 complete. Loss:  0.17765726149082184
batch 48 complete. Loss:  0.10285371541976929
batch 49 complete. Loss:  0.16085514426231384
batch 50 complete. Loss:  0.12417875230312347
batch 51 complete. Loss:  0.3295987844467163
batch 52 complete. Loss:  0.14273270964622498
batch 53 complete. Loss:  0.1438235342502594
batch 54 complete. Loss:  0.21680273115634918
batch 55 complete. Loss:  0.2288345843553543
batch 56 complete. Loss:  0.19796226918697357
batch 57 complete. Loss:  0.1463388055562

...

batch 181 complete. Loss:  0.019195500761270523
batch 182 complete. Loss:  0.08910372853279114
batch 183 complete. Loss:  0.07157435268163681
batch 184 complete. Loss:  0.022264841943979263
batch 185 complete. Loss:  0.01663903519511223
batch 186 complete. Loss:  0.013817119412124157
batch 187 complete. Loss:  0.030764950439333916
batch 188 complete. Loss:  0.024432379752397537
batch 189 complete. Loss:  0.029178794473409653
batch 190 complete. Loss:  0.016646448522806168
batch 191 complete. Loss:  0.020639777183532715
batch 192 complete. Loss:  0.04555830359458923
batch 193 complete. Loss:  0.022617504000663757
batch 194 complete. Loss:  0.01550272572785616
batch 195 complete. Loss:  0.015977229923009872
batch 196 complete. Loss:  0.019856223836541176
batch 197 complete. Loss:  0.028174590319395065
batch 198 complete. Loss:  0.027691910043358803
batch 199 complete. Loss:  0.022971797734498978
batch 200 complete. Loss:  0.01584327034652233
batch 201 complete. Loss:  0.01940581575036049

batch 354 complete. Loss:  0.032782625406980515
batch 355 complete. Loss:  0.02832995355129242
batch 356 complete. Loss:  0.024798259139060974
batch 357 complete. Loss:  0.01873948611319065
batch 358 complete. Loss:  0.026042625308036804
batch 359 complete. Loss:  0.028648819774389267
batch 360 complete. Loss:  0.0351032018661499
batch 361 complete. Loss:  0.07504221796989441
batch 362 complete. Loss:  0.053852636367082596
batch 363 complete. Loss:  0.012038737535476685
batch 364 complete. Loss:  0.010454870760440826
batch 365 complete. Loss:  0.026806462556123734
batch 366 complete. Loss:  0.013660338707268238
batch 367 complete. Loss:  0.031844042241573334
batch 368 complete. Loss:  0.012471869587898254
batch 369 complete. Loss:  0.025263674557209015
batch 370 complete. Loss:  0.019406002014875412
batch 371 complete. Loss:  0.05038627237081528
batch 372 complete. Loss:  0.08040334284305573
batch 373 complete. Loss:  0.026737118139863014
batch 374 complete. Loss:  0.019943254068493843

batch 527 complete. Loss:  0.08279125392436981
batch 528 complete. Loss:  0.03137415274977684
batch 529 complete. Loss:  0.025958213955163956
batch 530 complete. Loss:  0.03377465531229973
batch 531 complete. Loss:  0.04333442449569702
batch 532 complete. Loss:  0.03143757954239845
batch 533 complete. Loss:  0.02738676592707634
batch 534 complete. Loss:  0.0156564824283123
batch 535 complete. Loss:  0.023387577384710312
batch 536 complete. Loss:  0.028023039922118187
batch 537 complete. Loss:  0.029802465811371803
batch 538 complete. Loss:  0.027432376518845558
batch 539 complete. Loss:  0.015324774198234081
batch 540 complete. Loss:  0.0255103912204504
batch 541 complete. Loss:  0.016647953540086746
batch 542 complete. Loss:  0.056784145534038544
batch 543 complete. Loss:  0.0254665520042181
batch 544 complete. Loss:  0.02913171797990799
batch 545 complete. Loss:  0.041065189987421036
batch 546 complete. Loss:  0.014018239453434944
batch 547 complete. Loss:  0.015401182696223259
batch

batch 25 complete. Loss:  0.04675748944282532
batch 26 complete. Loss:  0.020644711330533028
batch 27 complete. Loss:  0.018635833635926247
batch 28 complete. Loss:  0.02501242235302925
batch 29 complete. Loss:  0.01414906419813633
batch 30 complete. Loss:  0.017175914719700813
batch 31 complete. Loss:  0.012842555530369282
batch 32 complete. Loss:  0.02498454600572586
batch 33 complete. Loss:  0.01914353482425213
batch 34 complete. Loss:  0.026240380480885506
batch 35 complete. Loss:  0.010543476790189743
batch 36 complete. Loss:  0.02025396004319191
batch 37 complete. Loss:  0.021527495235204697
batch 38 complete. Loss:  0.010715119540691376
batch 39 complete. Loss:  0.015984883531928062
batch 40 complete. Loss:  0.015766337513923645
batch 41 complete. Loss:  0.019159339368343353
batch 42 complete. Loss:  0.014187546446919441
batch 43 complete. Loss:  0.02279421128332615
batch 44 complete. Loss:  0.020698312669992447
batch 45 complete. Loss:  0.02029728703200817
batch 46 complete. Lo

KeyboardInterrupt: 

In [21]:
idx =101

#text=scaled_data['text'][idx]
text='''East Lansing Public Schools, home of Michigan State University (MSU) and a neighbor to the state capital, seeks a
dynamic individual to work in our highly diverse school district. This individual must love kids, have a passion for
their success, be a collaborative team player, deeply understand best practices for teaching and learning, and
engage all students in the learning process. Work in a district that has built six new elementary schools with stateof-the-art instructional and sustainable enhancements, has multiple partnerships with MSU, encourages
continuous professional growth and learning, and is highly supported by its families and community!

POSITION: Summer School Administrator (shared position)
LOCATION: Donley and Red Cedar Elementary Schools
SALARY: Stipend: $3,500
WORK YEAR: Weeks of June 24, July 8, July 15, Plus July 22 and July 25 - Work Days: M, T, W, TH 8:30 AM to
1:30 PM
START DATE: June 24, 2024
REPORTS TO: Assistant Superintendent
APPLICATION DEADLINE: March 15, 2024


GENERAL SUMMARY: Under the direct supervision of the Assistant Superintendent, and in accordance with the
established policies and procedures of the East Lansing Public School District, the Summer School Administrator is
responsible for the administration and oversight of the elementary summer school program. This person will be
sharing the administration with an elementary principal and working the weeks/days the elementary principal is not
working. The elementary summer school program will likely be run at two sites: Donley and Red Cedar, so travel
between sites will be necessary.'''
data = torch.tensor(nn_data[idx]).unsqueeze(0)

# inference
inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to('cuda')
outputs = model(**inputs, nn_data=data)
pred = scaler.inverse_transform(np.array([outputs.item()]))
print(pred)
#print(pred, scaler.inverse_transform(np.array([scaled_data['label'][idx]])))



[42675.0019354]


In [22]:
train_losses, val_losses

([0.40143253423549513,
  0.16957210713514576,
  0.1047804256418237,
  0.07134516571406965,
  0.05439718822362246,
  0.041425507995817394,
  0.034718237422682624,
  0.029447722020386546],
 [0.2210999514312434,
  0.15841861827486364,
  0.11680733535945768,
  0.105252427616592,
  0.10249069069660979,
  0.08509285335798235,
  0.07618940602877788,
  0.08059447902370487])

In [23]:
train_losses = []
with torch.no_grad():  # Disable gradient calculation for validation
    for batch in train_dataloader:
        input_ids = batch['input_ids'].to('cuda') 
        attention_mask = batch['attention_mask'].to('cuda')
        labels = batch['labels'].to('cuda')
        data = batch['nn_data'].to('cuda')
        for i in range(len(labels)):
            output = model(input_ids[i].unsqueeze(0), 
                           attention_mask=attention_mask[i].unsqueeze(0),
                           nn_data = data[i].unsqueeze(0))
            yhat = scaler.inverse_transform(np.array([output.squeeze(1).item()]))
            y=scaler.inverse_transform(np.array([labels[i].unsqueeze(0).item()]))
            loss = np.abs(yhat-y)
            
            train_losses.append(loss)
            #print(loss)


In [24]:
test_losses = []
with torch.no_grad():  # Disable gradient calculation for validation
    for batch in val_dataloader:
        input_ids = batch['input_ids'].to('cuda') 
        attention_mask = batch['attention_mask'].to('cuda')
        labels = batch['labels'].to('cuda')
        data = batch['nn_data'].to('cuda')
        for i in range(len(labels)):
            output = model(input_ids[i].unsqueeze(0), 
                           attention_mask=attention_mask[i].unsqueeze(0),
                           nn_data = data[i].unsqueeze(0))
            yhat = scaler.inverse_transform(np.array([output.squeeze(1).item()]))
            y=scaler.inverse_transform(np.array([labels[i].unsqueeze(0).item()]))
            loss = np.abs(yhat-y)
            
            test_losses.append(loss)
            
        


In [26]:
print("Mean training err: $" +  str(np.mean(train_losses)))
print("Median training err: $" + str(np.median(train_losses)))
print("Mean test err: $" + str(np.mean(test_losses)))
print("Median test err: $" + str(np.median(test_losses)))

Mean training err: $6472.70942511934
Median training err: $4943.678810362908
Mean test err: $9306.064277280737
Median test err: $5682.710841949112


In [ ]:
#del model
#torch.cuda.empty_cache()